# SUPERSMARTHOME


### 0: INSTALLING LIBRARIES

In [152]:
# !pip install llama_index chroma chromadb openai logging llama-index-experimental
# !pip install -r requirements.txt


### 0: IMPORTING LIBRARIES
Using pre-built class for directory reading.

In [153]:
import os
import sys
import logging
import chromadb
import pandas as pd
from IPython.display import Markdown, display
import re

from llama_index.core import StorageContext
from llama_index.core import Settings
from llama_index.core import SimpleDirectoryReader
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.embeddings.openai import OpenAIEmbedding


In [154]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### 1: IMPORTING OPENAPI KEY

In [155]:
# first, load and set openaikey from a txt file I stored it in
with open('oaikey.txt') as keyfile:
    oaikey = keyfile.read().strip()
    
os.environ["OPENAI_API_KEY"] = oaikey

#### 2: IMPORTING DATASET 

In [156]:
# File paths for each room
room_files = {
    'QDORO': './MQTT Client/Room MQTT Client/data/airQDORO.csv',
    'QFOYER': './MQTT Client/Room MQTT Client/data/airQFOYER.csv',
    'QHANS': './MQTT Client/Room MQTT Client/data/airQHANS.csv',
    'QMOMO': './MQTT Client/Room MQTT Client/data/airQMOMO.csv',
    'QRITA': './MQTT Client/Room MQTT Client/data/airQRITA.csv',
    'QROB': './MQTT Client/Room MQTT Client/data/airQROB.csv'
}

# Load each room's data into separate DataFrames
df_QDORO = pd.read_csv(room_files['QDORO'])
df_QFOYER = pd.read_csv(room_files['QFOYER'])
df_QHANS = pd.read_csv(room_files['QHANS'])
df_QMOMO = pd.read_csv(room_files['QMOMO'])
df_QRITA = pd.read_csv(room_files['QRITA'])
df_QROB = pd.read_csv(room_files['QROB'])

df_rooftop = pd.read_csv('./MQTT Client/Roof MQTT Client/data/pivoted_data.csv')


In [157]:
# Optionally, convert timestamp to datetime if needed
df_QDORO['timestamp'] = pd.to_datetime(df_QDORO['timestamp'])
df_QFOYER['timestamp'] = pd.to_datetime(df_QFOYER['timestamp'])
df_QHANS['timestamp'] = pd.to_datetime(df_QHANS['timestamp'])
df_QMOMO['timestamp'] = pd.to_datetime(df_QMOMO['timestamp'])
df_QRITA['timestamp'] = pd.to_datetime(df_QRITA['timestamp'])
df_QROB['timestamp'] = pd.to_datetime(df_QROB['timestamp'])

df_rooftop['timestamp'] = pd.to_datetime(df_rooftop['timestamp_utc'])

### 3: BUILDING PANDAS QUERY ENGINE

In [158]:
# Internal Rooms
query_engine_DORO = PandasQueryEngine(df=df_QDORO, verbose=True)
query_engine_FOYER = PandasQueryEngine(df=df_QFOYER, verbose=True)
query_engine_HANS = PandasQueryEngine(df=df_QHANS, verbose=True)
query_engine_MOMO = PandasQueryEngine(df=df_QMOMO, verbose=True)
query_engine_RITA = PandasQueryEngine(df=df_QRITA, verbose=True)
query_engine_ROB = PandasQueryEngine(df=df_QROB, verbose=True)

# External Rooms
query_engine_rooftop = PandasQueryEngine(df=df_rooftop, verbose=True)

### 4: QUERYING THE DATA

##### 4.1: ROOFTOP QUERYING

In [159]:
internal_external_choice = "external"   # "external", "internal"

# If the choice is internal, I need to set the Room Number.
# internal_room_choice = "ROOFTOP"   # "DORO", "FOYER", "HANS", "MOMO", "RITA", "ROB"
internal_room_choice = "ROOFTOP"

In [160]:
# Set query_engine_choice based on the internal/external choice and the room choice
if internal_external_choice == "internal":
    
    if internal_room_choice == "QDORO":
        query_engine_choice = query_engine_DORO
    elif internal_room_choice == "QFOYER":
        query_engine_choice = query_engine_FOYER
    elif internal_room_choice == "QHANS":
        query_engine_choice = query_engine_HANS
    elif internal_room_choice == "QMOMO":
        query_engine_choice = query_engine_MOMO
    elif internal_room_choice == "QRITA":
        query_engine_choice = query_engine_RITA
    elif internal_room_choice == "QROB":
        query_engine_choice = query_engine_ROB
    else:
        raise ValueError(f"Unknown internal room choice: {internal_room_choice}")

elif internal_external_choice == "external":
    
    query_engine_choice = query_engine_rooftop

else:
    raise ValueError(f"Unknown choice for internal_external_choice: {internal_external_choice}")


In [161]:
df_rooftop

,timestamp_utc,sensor_sn,Air Temperature ( °C),Atmospheric Pressure ( kPa),EC ( mS/cm),Gust Speed ( m/s),Max Air Temperature ( °C),Max Precip Rate ( mm/h),Min Air Temperature ( °C),Precipitation ( mm),RH Sensor Temp ( °C),Solar Radiation ( W/m²),Tilt Angle (°),VPD ( kPa),Vapor Pressure ( kPa),Wind Direction (°),Wind Speed ( m/s),timestamp
0,1719784800,A4100209,20.0,98.00,0.0,2.11,20.1,0.0,20.0,0.0,19.7,0.0,1.4,0.28,2.067,289.7,0.68,1970-01-01 00:00:01.719784800
1,1719785100,A4100209,20.1,98.00,0.0,2.48,20.1,0.0,20.0,0.0,19.8,0.0,1.4,0.29,2.060,302.9,0.75,1970-01-01 00:00:01.719785100
2,1719785400,A4100209,20.1,98.01,0.0,2.38,20.2,0.0,20.0,0.0,19.8,0.0,1.4,0.30,2.054,298.2,0.72,1970-01-01 00:00:01.719785400
3,1719785700,A4100209,20.1,98.01,0.0,1.80,20.2,0.0,20.0,0.0,19.9,0.0,1.3,0.30,2.047,292.3,0.85,1970-01-01 00:00:01.719785700
4,1719786000,A4100209,20.1,98.02,0.0,3.53,20.2,0.0,20.0,0.0,19.9,0.0,1.3,0.32,2.036,286.0,0.99,1970-01-01 00:00:01.719786000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,1719956400,A4100209,15.8,98.04,0.0,0.64,15.9,0.0,15.7,0.0,15.9,0.0,1.3,0.41,1.380,197.7,0.34,1970-01-01 00:00:01.719956400
573,1719956700,A4100209,15.7,98.04,0.0,0.54,15.8,0.0,15.6,0.0,15.8,0.0,1.4,0.40,1.378,194.1,0.24,1970-01-01 00:00:01.719956700
574,1719957000,A4100209,15.7,98.03,0.0,0.74,15.8,0.0,15.5,0.0,15.7,0.0,1.4,0.41,1.370,148.3,0.25,1970-01-01 00:00:01.719957000
575,1719957300,A4100209,15.6,98.03,0.0,1.05,15.7,NaN,15.5,0.0,15.7,0.0,1.3,0.41,1.362,172.4,0.49,1970-01-01 00:00:01.719957300


In [162]:
query = f"Please provide me the set of air temperature values during last day in {internal_room_choice}"

response = query_engine_choice.query(
    query,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df[df['timestamp_utc'] >= df['timestamp_utc'].max() - 86400]['Air Temperature ( °C)']
```
> Pandas Output: 288    16.3
289    16.3
290    16.3
291    16.2
292    16.3
       ... 
572    15.8
573    15.7
574    15.7
575    15.6
576    15.6
Name: Air Temperature ( °C), Length: 289, dtype: float64


In [163]:
# Assuming you have a Response object named 'response'
actual_response = response.response

# Now you can print or further process the actual response string
print(actual_response)

288    16.3
289    16.3
290    16.3
291    16.2
292    16.3
       ... 
572    15.8
573    15.7
574    15.7
575    15.6
576    15.6
Name: Air Temperature ( °C), Length: 289, dtype: float64


##### 4.2: DORO QUERYING

In [164]:
# Internal Rooms
query_engine_DORO = PandasQueryEngine(df=df_QDORO, verbose=True)
query_engine_FOYER = PandasQueryEngine(df=df_QFOYER, verbose=True)
query_engine_HANS = PandasQueryEngine(df=df_QHANS, verbose=True)
query_engine_MOMO = PandasQueryEngine(df=df_QMOMO, verbose=True)
query_engine_RITA = PandasQueryEngine(df=df_QRITA, verbose=True)
query_engine_ROB = PandasQueryEngine(df=df_QROB, verbose=True)

# External Rooms
query_engine_rooftop = PandasQueryEngine(df=df_rooftop, verbose=True)

In [165]:
internal_external_choice = "internal"   # "external", "internal"

# If the choice is internal, I need to set the Room Number.
internal_room_choice = "QRITA"   # "QDORO", "QFOYER", "QHANS", "QMOMO", "QRITA", "QROB"


In [166]:
# Set query_engine_choice based on the internal/external choice and the room choice
if internal_external_choice == "internal":
    
    if internal_room_choice == "QDORO":
        query_engine_choice = query_engine_DORO
    elif internal_room_choice == "QFOYER":
        query_engine_choice = query_engine_FOYER
    elif internal_room_choice == "QHANS":
        query_engine_choice = query_engine_HANS
    elif internal_room_choice == "QMOMO":
        query_engine_choice = query_engine_MOMO
    elif internal_room_choice == "QRITA":
        query_engine_choice = query_engine_RITA
    elif internal_room_choice == "QROB":
        query_engine_choice = query_engine_ROB
    else:
        raise ValueError(f"Unknown internal room choice: {internal_room_choice}")

elif internal_external_choice == "external":
    
    query_engine_choice = query_engine_rooftop

else:
    raise ValueError(f"Unknown choice for internal_external_choice: {internal_external_choice}")


In [167]:
timestep_request = 'and the timestep'
query = f"Please provide me all the health {timestep_request} values during first day in the room {internal_room_choice}."
query = query + ' Provide also the timestep column associated to the result.'
print(query)
response = query_engine_choice.query(
    query,
)

Please provide me all the health and the timestep values during first day in the room QRITA. Provide also the timestep column associated to the result.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df[df['timestamp'].dt.date == df['timestamp'].dt.date.iloc[0]][['health', 'timestamp']]
```
> Pandas Output:     health                  timestamp
0      333 1970-01-01 0

In [168]:
# Assuming you have a Response object named 'response'
actual_response = response.response

# Now you can print or further process the actual response string
print(actual_response)


    health                  timestamp
0      333 1970-01-01 00:28:45.009675
1      337 1970-01-01 00:28:45.009707
2      338 1970-01-01 00:28:45.009715
3      338 1970-01-01 00:28:45.009723
4      338 1970-01-01 00:28:45.009731
..     ...                        ...
87     341 1970-01-01 00:28:45.010416
88     343 1970-01-01 00:28:45.010452
89     343 1970-01-01 00:28:45.010460
90     343 1970-01-01 00:28:45.010460
91     342 1970-01-01 00:28:45.010468

[92 rows x 2 columns]


In [169]:
import pandas as pd
import re

# Split the string into lines
lines = actual_response.split('\n')

# Filter out lines that contain '..', 'health', or are empty
filtered_lines = [line for line in lines if line.strip() and '..' not in line and 'health' not in line]

# Use regex to extract health and timestamp values
parsed_data = []
for line in filtered_lines:
    match = re.match(r'\s*(\d+)\s+(\d+)\s+([\d-]+ [\d:.]+)', line)
    if match:
        index = int(match.group(1))  # The first number (index)
        health = int(match.group(2))  # The second number (health)
        timestamp = match.group(3)  # The timestamp string
        parsed_data.append((index, health, timestamp))

# Convert to DataFrame
df_response = pd.DataFrame(parsed_data, columns=['Index', 'Health', 'Timestamp'])

# Remove the last row
df_response = df_response.iloc[:-1]

# Convert 'Timestamp' to datetime
df_response['Timestamp'] = pd.to_datetime(df_response['Timestamp'])

# Set 'Index' as the index of the DataFrame
df_response.set_index('Index', inplace=True)

# Display the resulting DataFrame
print(df_response)

       Health                  Timestamp
Index                                   
0         333 1970-01-01 00:28:45.009675
1         337 1970-01-01 00:28:45.009707
2         338 1970-01-01 00:28:45.009715
3         338 1970-01-01 00:28:45.009723
4         338 1970-01-01 00:28:45.009731
87        341 1970-01-01 00:28:45.010416
88        343 1970-01-01 00:28:45.010452
89        343 1970-01-01 00:28:45.010460
90        343 1970-01-01 00:28:45.010460


In [170]:
from openai import OpenAI
client = OpenAI(api_key=oaikey)
data = None
chat_completion = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{'role': 'user', 
               'content': f'Here it is the user query: {query}.'
               'Generate python code that prints a line chart of the full data. The chart should include timestep on the x axis and the other value on the y-axis'
               #'Put a placeholder for the data variable like {df_response}.'
               'Only print the python code. Do not include comments.'
               'Do not output any other text before or after the code.'}])

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [171]:
print(chat_completion.choices[0].message.content)

```python
import pandas as pd
import matplotlib.pyplot as plt

df_response = pd.DataFrame()  # Placeholder for the actual data

first_day_data = df_response[(df_response['room'] == 'QRITA') & (df_response['date'] == df_response['date'].min())]
plt.plot(first_day_data['timestep'], first_day_data['health'])
plt.xlabel('Timestep')
plt.ylabel('Health')
plt.title('Health vs Timestep in Room QRITA - First Day')
plt.show()
```


In [172]:
# new_code = (chat_completion.choices[0].message.content[9:-3].replace('{df_response}', 
                                                            #   f"{df_response[df_response.columns.tolist()].to_dict()}"))

IndentationError: unexpected indent (1061049681.py, line 2)

In [173]:
print(new_code)


import pandas as pd
import matplotlib.pyplot as plt

data = {}
df = pd.DataFrame(data)
df['timestamp'] = pd.to_datetime(df['timestamp'])
first_day = df[df['timestamp'].dt.day == 1]
plt.plot(first_day['timestamp'], first_day['health'])
plt.xlabel('Timestep')
plt.ylabel('Health')
plt.title('Health over time in QRITA')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



In [174]:
exec(new_code)

KeyError: 'timestamp'